Q1. Ninjacart: CV Classification
Unsolved
warning icon
Your business case was due on 10 Oct 2024, 6:30 PM.Your score will be reduced by 70.
feature icon
Using hints except Complete Solution is Penalty free now
Use Hint
Problem Statement

Ninjacart is India's largest fresh produce supply chain company. They are pioneers in solving one of the toughest supply chain problems of the world by leveraging innovative technology. They source fresh produce from farmers and deliver them to businesses within 12 hours. An integral component of their automation process is the development of robust classifiers which can distinguish between images of different types of vegetables, while also correctly labeling images that do not contain any one type of vegetable as noise.

As a starting point, ninjacart has provided us with a dataset scraped from the web which contains train and test folders, each having 4 sub-folders with images of onions, potatoes, tomatoes and some market scenes. We have been tasked with preparing a multiclass classifier for identifying these vegetables. The dataset provided has all the required images to achieve the task.

Dataset Link

https://drive.google.com/file/d/1clZX-lV_MLxKHSyeyTheX5OCQtNCUcqT/view?usp=sharing

Context

This dataset contains images of the following food items: noise-Indian market and images of vegetables- onion, potato and tomato.

Data Collection

The images in this dataset were scraped from Google.

Content

This dataset contains a folder train, which has a total of 3135 images, split into four folders as follows:

Tomato : 789

Potato : 898

Onion : 849

Indian market : 599

This dataset contains another folder test which has a total of 351 images, split into four folders

Tomato : 106

potato : 83

onion : 81

Indian market : 81

Inspiration

The objective is to develop a program that can recognize the vegetable item(s) in a photo and identify them for the user.

Concepts Tested:

Dataset Preparation & Visualization

CNN models

Implementing Callbacks

Deal with Overfitting

Transfer Learning

Process:

There are numerous libraries that can be used to work with images. Importing the libraries we feel most confident with is our first step: TensorFlow, keras, matplotlib, opencv, seaborn etc

Download the Data (you can use gdown) with the provided Dataset Link and unzip it

Visualize the data, use the dataset directory to create a list containing all the image paths in the training folder. You can use matplotlib or tensorflow to plot a grid sample of the images you fetched from the list of image paths.

Plot a few of the images of each class to check their dimensions. [Note that the images are not all of uniform dimensions]

Verify the count of images in each train and test folder by plotting histogram .

Check each folder to see if the number of images matches the reported number.

Split the dataset to a train and validation set.

The provided data does not contain separate training and validation folders. For us to do hyperparameter tuning of our models, it is important to divide the dataset into an 80-20 split for training and validation respectively.

Before fitting data to our model, we must make sure that each image is square-shaped so that we may resize it to the required dimensions and also perform rescaling which will rescale the inputs between 0-1 by dividing each value by 255.

Use a model of your choice (could be vgg, resnet and mobilenet) and train it with an appropriate batch size.

Using the pretrained weights of popular networks is a great way to do transfer learning, since the size of our original dataset is small.

Obtain the testing accuracy to see how well your model generalizes.

Compare and check the performance of multiple models using a confusion matrix.

Implement a TensorBoard callback to log each of our model metrics for each model during the training process. [ recommended google colab or jupyter notebook]

Plot the train/valid accuracy and loss graph for each model.

If the model is overfitting , Try to tune your model by applying - :

BatchNormalization and Dropout

Callbacks : EarlyStopping, ModelCheckpoint and TensorBoard callback

Data Augmentation

Evaluation Criteria (100 points)

Importing the dataset and doing usual exploratory analysis steps like checking the structure & characteristics of the data (10 points)

Exploratory Data Analysis. (20 points)

Plotting class distribution & Visualizing Image dimensions with their plots[10]

Splitting the dataset into train, validation, and test set[10]

Creating model architecture and training (50 points)

Defining the CNN Classifier model from scratch[10]

Improving Baseline CNN to reduce overfitting[10]

Implementing Callbacks while training the model[10]

Finetune pretrained models such as VGG, ResNet and MobileNet[10]

Plotting the model training metrics and confusion matrix[10]

Testing your best model so far(20 points)

Testing on the test set & Random image samples prediction[10]

Summary & Insights [10]

In [1]:
import os
from pathlib import Path
import glob
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt


Setting paramteres

In [2]:
DATA_ROOT= Path('/content/view?usp=sharing.1')
TRAIN_DIR = DATA_ROOT/'train'
TEST_DIR= DATA_ROOT/ 'test'
IMAGE_SIZE=(256,256)
BATCH_SIZE=64
SEED=2022

# **Loading the Data**

Unzipping downloaded data

In [3]:
if not os.path.isdir('/content/view?usp=sharing.1'):
  os.system('unzip/content/view?usp=sharing.1')

In [4]:
!unzip '/content/view?usp=sharing.1'

unzip:  cannot find or open /content/view?usp=sharing.1, /content/view?usp=sharing.1.zip or /content/view?usp=sharing.1.ZIP.

No zipfiles found.


Visualizing the Data

In [5]:
train_folder=
images=[]
for folder in os.listdir(train_folder):
  for image in os.listdir(train_folder+'/'+folder):
    images.append(os.path.join(train_folder, folder, image))

fig=plt.figure(1,figsize=(15,9))
fig.suptitle('Data overview', fontsize=25)
plt.axis('off')
n=0
for i in range(16):
  n==1
  random_img= random.choice(images)
  imgs= tf.keras.utlis.load_img(random_img)
  plt.subplot(4,4,n)
  plt.axis('off')
  plt.inshow(imgs)

plt.show()

SyntaxError: invalid syntax (ipython-input-3396057325.py, line 1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install required packages
!pip install gdown

# Download the dataset
!gdown 1clZX-lV_MLxKHSyeyTheX5OCQtNCUcqT

# Unzip the downloaded file
import zipfile
with zipfile.ZipFile('/content/ninjacart_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

import os
from pathlib import Path
import glob
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

# Setting parameters
DATA_ROOT = Path('/content/ninjacart_data')
TRAIN_DIR = DATA_ROOT / 'train'
TEST_DIR = DATA_ROOT / 'test'
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32
SEED = 2022

# Set random seeds for reproducibility
tf.random.set_seed(SEED)
np.random.seed(SEED)

# 1. Data Loading and Exploration

print("Dataset structure:")
print(f"Train directory exists: {TRAIN_DIR.exists()}")
print(f"Test directory exists: {TEST_DIR.exists()}")

# Check class distribution
def count_images_per_class(directory):
    class_counts = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            num_images = len([f for f in os.listdir(class_path)
                            if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            class_counts[class_name] = num_images
    return class_counts

train_counts = count_images_per_class(TRAIN_DIR)
test_counts = count_images_per_class(TEST_DIR)

print("\nTraining set distribution:")
for class_name, count in train_counts.items():
    print(f"{class_name}: {count}")

print("\nTest set distribution:")
for class_name, count in test_counts.items():
    print(f"{class_name}: {count}")

# 2. Data Visualization

def visualize_samples(directory, num_samples=16):
    fig, axes = plt.subplots(4, 4, figsize=(15, 12))
    fig.suptitle('Data Overview - Sample Images from Each Class', fontsize=20)
    axes = axes.ravel()

    all_images = []
    class_names = []

    for class_idx, class_name in enumerate(sorted(os.listdir(directory))):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            images = [f for f in os.listdir(class_path)
                     if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            for i in range(4):
                if i < len(images):
                    img_path = os.path.join(class_path, images[i])
                    img = Image.open(img_path)
                    axes[class_idx*4 + i].imshow(img)
                    axes[class_idx*4 + i].set_title(f'{class_name}\n{img.size}')
                    axes[class_idx*4 + i].axis('off')

    plt.tight_layout()
    plt.show()

print("\nVisualizing training samples:")
visualize_samples(TRAIN_DIR)

# Plot class distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Training distribution
ax1.bar(train_counts.keys(), train_counts.values())
ax1.set_title('Training Set Class Distribution')
ax1.set_xlabel('Classes')
ax1.set_ylabel('Number of Images')
ax1.tick_params(axis='x', rotation=45)

# Test distribution
ax2.bar(test_counts.keys(), test_counts.values())
ax2.set_title('Test Set Class Distribution')
ax2.set_xlabel('Classes')
ax2.set_ylabel('Number of Images')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 3. Data Preparation and Preprocessing

# Create data generators with data augmentation for training
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    seed=SEED
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"\nClasses: {train_generator.class_indices}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# 4. Model Building - CNN from Scratch
def create_cnn_model():
    model = keras.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.25),

        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.25),

        # Third Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.25),

        # Fourth Convolutional Block
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.25),

        # Classifier
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4, activation='softmax')  # 4 classes
    ])

    return model

# Create and compile model
cnn_model = create_cnn_model()
cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("CNN Model Summary:")
cnn_model.summary()

# 5. Callbacks Implementation

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import datetime

# Create callbacks
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_cnn_model.h5', save_best_only=True, verbose=1),
    ReduceLROnPlateau(factor=0.2, patience=5, verbose=1),
    TensorBoard(log_dir=f"logs/cnn/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}")
]

# 6. Model Training

print("\nTraining CNN Model...")
history = cnn_model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

#7. Model Evaluation and Visualization

# Plot training history
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()

    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history)

# Evaluate on test set
print("\nEvaluating on Test Set...")
test_loss, test_accuracy = cnn_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")


# 8. Confusion Matrix and Classification Report

# Get predictions
y_pred = cnn_model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

class_names = list(test_generator.class_indices.keys())

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))

# =============================================================================
# 9. Transfer Learning with Pre-trained Models
# =============================================================================

def create_transfer_learning_model(base_model, model_name):
    base_model.trainable = False

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(4, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Try different pre-trained models
pre_trained_models = {
    'MobileNetV2': tf.keras.applications.MobileNetV2(
        input_shape=(256, 256, 3),
        include_top=False,
        weights='imagenet'
    ),
    'VGG16': tf.keras.applications.VGG16(
        input_shape=(256, 256, 3),
        include_top=False,
        weights='imagenet'
    ),
    'ResNet50': tf.keras.applications.ResNet50(
        input_shape=(256, 256, 3),
        include_top=False,
        weights='imagenet'
    )
}

# Train and compare different models
model_results = {}

for model_name, base_model in pre_trained_models.items():
    print(f"\n{'='*50}")
    print(f"Training {model_name}")
    print(f"{'='*50}")

    model = create_transfer_learning_model(base_model, model_name)

    # Callbacks for transfer learning
    tl_callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint(f'best_{model_name.lower()}_model.h5', save_best_only=True),
        ReduceLROnPlateau(factor=0.2, patience=3)
    ]

    # Train the model
    history_tl = model.fit(
        train_generator,
        epochs=30,
        validation_data=validation_generator,
        callbacks=tl_callbacks,
        verbose=1
    )

    # Evaluate on test set
    test_loss, test_accuracy = model.evaluate(test_generator)
    model_results[model_name] = test_accuracy

    print(f"{model_name} Test Accuracy: {test_accuracy:.4f}")

# =============================================================================
# 10. Results Comparison and Summary
# =============================================================================

print("\n" + "="*60)
print("MODEL COMPARISON SUMMARY")
print("="*60)

# Compare all models
print("\nModel Performance Comparison:")
for model_name, accuracy in model_results.items():
    print(f"{model_name}: {accuracy:.4f}")

best_model_name = max(model_results, key=model_results.get)
print(f"\nBest Model: {best_model_name} with accuracy: {model_results[best_model_name]:.4f}")

# Load the best model for final predictions
if model_results[best_model_name] > test_accuracy:
    best_base_model = pre_trained_models[best_model_name]
    best_model = create_transfer_learning_model(best_base_model, best_model_name)
    best_model.load_weights(f'best_{best_model_name.lower()}_model.h5')
else:
    best_model = cnn_model
    best_model.load_weights('best_cnn_model.h5')

# Final evaluation with best model
final_test_loss, final_test_accuracy = best_model.evaluate(test_generator)
print(f"\nFinal Best Model Test Accuracy: {final_test_accuracy:.4f}")

# =============================================================================
# 11. Prediction on Random Samples
# =============================================================================

def predict_random_samples(model, num_samples=8):
    # Get random test samples
    sample_indices = np.random.choice(len(test_generator.filenames), num_samples, replace=False)

    fig, axes = plt.subplots(2, 4, figsize=(15, 8))
    axes = axes.ravel()

    for i, idx in enumerate(sample_indices):
        # Get image path
        img_path = os.path.join(TEST_DIR, test_generator.filenames[idx])
        true_class = test_generator.filenames[idx].split('/')[0]

        # Load and preprocess image
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMAGE_SIZE)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Make prediction
        prediction = model.predict(img_array)
        predicted_class = class_names[np.argmax(prediction)]
        confidence = np.max(prediction)

        # Plot
        axes[i].imshow(img)
        axes[i].set_title(f'True: {true_class}\nPred: {predicted_class}\nConf: {confidence:.2f}')
        axes[i].axis('off')

        # Color code based on correctness
        color = 'green' if true_class == predicted_class else 'red'
        for spine in axes[i].spines.values():
            spine.set_edgecolor(color)
            spine.set_linewidth(3)

    plt.tight_layout()
    plt.show()

print("\nPredicting on random test samples:")
predict_random_samples(best_model)

# =============================================================================
# 12. Insights and Summary
# =============================================================================

print("\n" + "="*60)
print("KEY INSIGHTS AND SUMMARY")
print("="*60)

print("\n1. DATA INSIGHTS:")
print(f"   - Total training images: {sum(train_counts.values())}")
print(f"   - Total test images: {sum(test_counts.values())}")
print(f"   - Number of classes: {len(train_counts)}")
print(f"   - Class distribution in training: {train_counts}")

print("\n2. MODEL PERFORMANCE:")
print(f"   - CNN from scratch accuracy: {test_accuracy:.4f}")
for model_name, acc in model_results.items():
    print(f"   - {model_name} accuracy: {acc:.4f}")

print("\n3. RECOMMENDATIONS:")
print("   - Data augmentation helps prevent overfitting")
print("   - Transfer learning generally performs better on small datasets")
print("   - Regularization techniques (Dropout, BatchNorm) are essential")
print("   - Class imbalance should be addressed for better performance")

print("\n4. POTENTIAL IMPROVEMENTS:")
print("   - Collect more balanced data")
print("   - Try ensemble methods")
print("   - Fine-tune pre-trained models")
print("   - Use class weights for imbalanced data")
print("   - Experiment with different architectures")

## Key Insights and Recommendations

Based on the analysis of the Ninjacart dataset and the training of different image classification models, here are some key insights and recommendations:

**Key Insights:**

*   **Dataset Characteristics:** The dataset contains images of four classes: 'Indian market' (noise), 'Onion', 'Potato', and 'Tomato'. The distribution of images across these classes is relatively balanced in both the training and test sets.
*   **Image Variability:** The images in the dataset have varying dimensions, highlighting the need for resizing and preprocessing before feeding them into a CNN model.
*   **CNN Model Performance:** The custom CNN model from scratch provided a baseline performance.
*   **Transfer Learning Effectiveness:** Pre-trained models like MobileNetV2, VGG16, and ResNet50, when used for transfer learning, generally outperform the custom CNN model, demonstrating the power of leveraging features learned on large datasets.
*   **Impact of Data Augmentation and Regularization:** Techniques like data augmentation, Batch Normalization, and Dropout were crucial in improving model generalization and reducing overfitting, especially with a relatively small dataset.
*   **Confusion Matrix and Classification Report:** These tools provide a detailed breakdown of model performance per class, highlighting potential areas where the model struggles (e.g., misclassifying certain classes).

**Recommendations:**

*   **Prioritize Transfer Learning:** Given the dataset size, transfer learning with pre-trained models is a highly effective approach. Experiment with fine-tuning the later layers of the pre-trained models for potentially better performance.
*   **Address Class Imbalance (if any):** While the current dataset is relatively balanced, in real-world scenarios, addressing class imbalance using techniques like weighted loss or oversampling/undersampling is important.
*   **Hyperparameter Tuning:** Further optimize the hyperparameters of the chosen model (e.g., learning rate, optimizer, number of layers in the classification head) for better performance.
*   **Explore Other Architectures:** Consider experimenting with other state-of-the-art pre-trained models or different custom CNN architectures.
*   **Collect More Data:** The performance of deep learning models generally improves with more data. If possible, collecting a larger and more diverse dataset would be beneficial.
*   **Ensemble Methods:** Combining predictions from multiple models can often lead to improved accuracy and robustness.
*   **Visualize Misclassifications:** Analyze the images that the model misclassifies to understand the reasons behind the errors and identify areas for improvement (e.g., poor image quality, confusing examples).

These insights and recommendations provide a roadmap for further improving the vegetable classification model for Ninjacart.